In [1]:
import pandas as pd
import os

In [2]:
"""Functions Related to Tranfering Data to Excel Sheet"""
def writeDataframe6(wbpath,df,sheetname,dfcols,xlcols,indexcol):    
    wb=opxl.load_workbook(wbpath)
    wsheet=wb[sheetname]    
    for index,row in df.iterrows():        
        myrow=int(row[indexcol])
        for idf,ixl in zip(dfcols,xlcols):
            wsheet.cell(row=myrow,column=ixl).value=row[idf]               
    wb.save(wbpath)
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
"""Packagewise and Structurewise Summation"""
def packageWiseSum(package_df,projection_df):
    plist=list(package_df['Package_No'])
    for index,row in projection_df.iterrows():
        pname=row['Package']
        row_index=plist.index(pname)
        package_df.iloc[row_index,1]=package_df.iloc[row_index,1]+row['VDW_30_06_2020']
        package_df.iloc[row_index,2]=package_df.iloc[row_index,2]+row['VDW_30_06_2021']
    for index,row in package_df.iterrows():
        package_df.iloc[index,4]= package_df.iloc[index,2]- package_df.iloc[index,3]
    return package_df
def structureWiseSum(structure_df,projection_df):
    plist=list(structure_df['Code'])
    for index,row in projection_df.iterrows():
        pname=row['Structure Code']
        row_index=plist.index(pname)
        structure_df.iloc[row_index,2]=structure_df.iloc[row_index,2]+row['VDW_30_06_2020']
        structure_df.iloc[row_index,3]=structure_df.iloc[row_index,3]+row['VDW_30_06_2021']
    for index,row in structure_df.iterrows():
        structure_df.iloc[index,5]= structure_df.iloc[index,3]- structure_df.iloc[index,4]
    return structure_df

In [4]:
def buildPackageWisePaymentMatrix(package_df):
    #Package_No Payment_30_06_2020
    package=list(package_df['Package_No'])
    paid=list(package_df['Payment_30_06_2020'])
    zero_list=[0 for x in package]
    data={'Package_No':package,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020':paid,'Payment_30_06_2021':zero_list}
    package_payment_df=pd.DataFrame(data)
    return package_payment_df
def buildStructureWisePaymentMatrix(structure_df):
    #DPP Item	Initial	Code
    structure_name=list(structure_df['DPP Item'])
    structure_code=list(structure_df['Code'])
    #Package_No Payment_30_06_2020    
    zero_list=[0 for x in structure_name]
    data={'DPP_Item':structure_name,'Code':structure_code ,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020':zero_list,'Payment_30_06_2021':zero_list}
    structure_payment_df=pd.DataFrame(data)
    return structure_payment_df

    
def buildVWDMatrix(projection_df):
    #Component	Unit	Qnty	Cost	Present_Progress	Projected_progress
    #Package
    #Structure Code
    stc=list(projection_df['Structure Code'])
    packages=list(projection_df['Package'])
    components=list(projection_df['Component'])
    units=list(projection_df['Unit'])
    quantity=list(projection_df['Qnty'])
    cost=list(projection_df['Cost'])
    present_prog=list(projection_df['Present_Progress'])
    projected_prog=list(projection_df['Projected_progress'])
    zero_list=[0 for x in components ]
    data={'Package':packages,'Structure Code':stc,'Component':components,'Unit':units,'Qnty':quantity,'Cost':cost,
          'Present_Progress':present_prog,'Projected_progress':projected_prog,
          'VDW_30_06_2020': zero_list,'VDW_30_06_2021':zero_list,'Work_Done_20_21':zero_list,'Payment_30_06_2020': zero_list,
          'Payment_30_06_2020':zero_list}
    vwd_mat=pd.DataFrame(data)
    for index,row in vwd_mat.iterrows():
        vwd_mat.iloc[index,8]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,6]
        vwd_mat.iloc[index,9]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,7]
        vwd_mat.iloc[index,10]=vwd_mat.iloc[index,9]-vwd_mat.iloc[index,8]
    return vwd_mat
    


In [5]:
myfolder=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\RADP Preparations'
input_file_name=os.path.join(myfolder,'Projecttion_input.xlsx')
out_file_name=os.path.join(myfolder,'Projecttion_output.xlsx')
myframes=[]
mynames=[]
sheetName="Structure_Type"
structure_type_df=pd.read_excel(input_file_name,sheet_name=sheetName)
structure_type_df.fillna(0,inplace=True)
structure_type_df
myframes.append(structure_type_df)
mynames.append("Structure_Type")
"""Projection Info"""
sheetName="Kishoregnj"
projection_info_df=pd.read_excel(input_file_name,sheet_name=sheetName)
projection_info_df.fillna(0,inplace=True)
projection_info_df
myframes.append(projection_info_df)
mynames.append("Projection_Info")
sheetName="Packages"
package_name_df=pd.read_excel(input_file_name,sheet_name=sheetName)
package_name_df.fillna(0,inplace=True)
myframes.append(package_name_df)
mynames.append("Package_Name")
vwd_df=buildVWDMatrix(projection_info_df)
myframes.append(vwd_df)
mynames.append("Value_of_Work_Done")
package_payment_df=buildPackageWisePaymentMatrix(package_name_df)
package_payment_df=packageWiseSum(package_payment_df,vwd_df)
myframes.append(package_payment_df)
mynames.append("Package_wise_payment")
"""Building DPP Itemwise Cost"""
structure_payment_df=buildStructureWisePaymentMatrix(structure_type_df)
structure_payment_df=structureWiseSum(structure_payment_df,vwd_df)
myframes.append(structure_payment_df)
mynames.append("DPP_Itemwise_payment")
saveDataFrame(myframes,out_file_name,mynames)



In [6]:
projection_info_df

,Package Name,Component,Unit,Qnty,Cost,Present_Progress,Projected_progress,Package,Haor Type,Structure Code,Earth Volume,Hoar Code,District Code
0,BWDB/Kish/HFMLIP/ PW-01,Betal Doba from KM 1.315 to KM 3.466 =2.151 KM,Km,2.151,40.940000,1.0,1.0,KISH/PW-01,Rehab,8,0.0,R-2,KISH
1,BWDB/Kish/HFMLIP/ PW-01,Alalia-bahadia Khal from KM 1.325 to KM 2.545=...,Km,1.220,35.880000,1.0,1.0,KISH/PW-01,Rehab,8,0.0,R-1,KISH
2,BWDB/Kish/HFMLIP/ PW-01,katakhali khal from KM 0.000 to KM 1.550=1.550 KM,Km,1.550,19.180000,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
3,BWDB/Kish/HFMLIP/ PW-01,Ganakkhali Khal from KM 0.000 to KM 5.625=5.62...,Km,5.625,309.800000,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
4,BWDB/Kish/HFMLIP/ PW-01,kali Khal from KM 0.00 to KM 1.000=1.000 KM,Km,1.000,45.950000,0.0,1.0,KISH/PW-01,Rehab,8,0.0,R-3,KISH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,BWDB/Kish/HFMLIP/ PW-26 Sonai Haor,Construction of sub-mergible embankment in bet...,Km,0.540,22.716098,0.0,0.0,KISH/PW-26,New,11,0.0,N-9,KISH
137,BWDB/Kish/HFMLIP/ PW-26 Sonai Haor,Re-excavation of Khal Phulesswarikhal from km ...,Km,3.960,71.526410,0.0,0.0,KISH/PW-26,New,7,0.0,N-9,KISH
138,BWDB/Kish/HFMLIP/ PW-26 Sonai Haor,GoalkhaliKhal from km 0.500 to km 2.600 = 2.1...,Km,2.100,12.722359,0.0,0.0,KISH/PW-26,New,7,0.0,N-9,KISH
139,BWDB/Kish/HFMLIP/ PW-26 Sonai Haor,KalnaKhal from km 0.800 to km 1.940 = 1.140 km,Km,1.140,16.387824,0.0,0.0,KISH/PW-26,New,7,0.0,N-9,KISH
